## Sea Ice Prediction - MLR+LSTM

In [ ]:
from numpy.random import seed
seed(1)

In [ ]:
#Install latest attention package
pip install attention

### Initial Setup

In [ ]:
import os
import math
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from attention import Attention
from keras.layers import Dense, Dropout
from keras.layers import LSTM,TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint


## Loading Combined Data 1979-2021

Features:
'wind_10m', 'specific_humidity', 'LW_down', 'SW_down', 'rainfall', 'snowfall' 'sst', 't2m', 'surface_pressure','sea_ice_extent'




In [ ]:
df = pd.read_csv('.../Arctic_domain_mean_monthly_1979_2021.csv')
df = df.drop(['Date'],axis=1)
data = np.array(df)

### Train a Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

y = data[:,-1] #assigning last column to be target variable
x = data[:,:] #dropping last column from features

model = LinearRegression()
model.fit(x, y)

lr_data = model.predict(x)
print(lr_data.shape)

(512,)


In [ ]:
lr= lr_data.reshape(len(lr_data),1)
print(lr.shape)

(512, 1)


Adding LR predictions as additional feature in LSTM dataset

#### Adding a Lag to Y values
Here lag = 1 - 3 months


In [ ]:
data = np.concatenate((data,lr),axis=1)
print(data.shape)

(512, 11)


In [ ]:
#Adding a lag to monthly targets
lag = 3
#test_data = data[-2:,:,:]
target = data[lag:,-1]
data = data[:-lag,:]

print(data.shape)
print(target.shape)


(509, 11)
(509,)


### Train Validation Split

LSTM network expects the input data to be provided with a specific array structure in the form of: [samples, time steps, features]. We load the csv file and only retain the feature and target columns. The features and target are stored in separate np arrays.

In [ ]:
# Sequential split train:val data 

LEN_DATA = len(data) #total number of pixels

NUM_TRAIN = LEN_DATA - (24+6) #reserve last 2.5 years for testing
NUM_VALID = LEN_DATA - NUM_TRAIN

print('LEN_DATA:',LEN_DATA)
print('NUM_TRAIN:',NUM_TRAIN)
print('NUM_VALID:',NUM_VALID)

x_train = data[0:NUM_TRAIN]
x_valid = data[NUM_TRAIN:]

#split features and labels
y_train=target[:NUM_TRAIN] #target is last column i-e sea-ice
y_valid=target[NUM_TRAIN:] #target is last column i-e sea-ice

LEN_DATA: 509
NUM_TRAIN: 479
NUM_VALID: 30


In [ ]:
print('x_train.shape:',x_train.shape)
print('y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape)
print('y_valid.shape:',y_valid.shape)

x_train.shape: (479, 11)
y_train.shape: (479,)
x_valid.shape: (30, 11)
y_valid.shape: (30,)


### Reshaping Input and Target Features

In [ ]:
# convert an array of values into a dataset matrix
def reshape_features(dataset, timesteps=1):
    print(dataset.shape)
    X = dataset.reshape((int(dataset.shape[0]/timesteps)), timesteps, dataset.shape[1])
    return X

### Normalization


In [ ]:
# normalize the features

scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train) 
x_valid = scaler_f.transform(x_valid) 
#test_data = scaler_f.transform(forecast)

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,1)) #reshaping to 2d for standard scaling
y_valid = scaler_l.transform(y_valid.reshape(-1,1)) #reshaping to 2d for standard scaling


In [ ]:
#Reshaping data to 3D for modeling
timesteps = 1
x_train = reshape_features(x_train, timesteps) # reshaping to 3d for model
x_valid = reshape_features(x_valid, timesteps) # reshaping to 3d for model

(479, 11)
(30, 11)


In [ ]:
print('x_train.shape:',x_train.shape)
print('y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape)
print('y_valid.shape:',y_valid.shape)

x_train.shape: (479, 1, 11)
y_train.shape: (479, 1)
x_valid.shape: (30, 1, 11)
y_valid.shape: (30, 1)


## LSTM Network

In [ ]:
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model

timestep = timesteps
features = 11

model_input = Input(shape=(timestep,features))
x = LSTM(64, return_sequences=True)(model_input)
x = Dropout(0.2)(x)
x = LSTM(32, return_sequences=True)(x)
x = LSTM(16, return_sequences=True)(x)
x = Attention(32)(x)
#x = Dropout(0.2)(x)
x = Dense(32)(x)
x = Dense(16)(x)
x = Dense(1)(x)
model = Model(model_input, x)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 11)]           0         
                                                                 
 lstm (LSTM)                 (None, 1, 64)             19456     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 32)             12416     
                                                                 
 lstm_2 (LSTM)               (None, 1, 16)             3136      
                                                                 
 attention (Attention)       (None, 32)                1280      
                                                                 
 dense (Dense)               (None, 32)                1056  

### Compiling the Network and Fitting Model

In [ ]:
#Compiling the network
model.compile(loss='mean_squared_error', optimizer='adam')
checkpoint_path='./testmodel.h5'
keras_callbacks   = [
      EarlyStopping(monitor='val_loss', patience=60, mode='min', min_delta=0.001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
]


In [ ]:
history=model.fit(x_train, y_train, epochs=500, batch_size=12, verbose=2, validation_split =0.33, shuffle=True,callbacks=keras_callbacks)

### Model Predictions

In [ ]:
trainPred = model.predict(x_train)
testPred = model.predict(x_valid)

In [ ]:
print(testPred.shape)
print(trainPred.shape)

(30, 1)
(479, 1)


In [ ]:
# invert scaling for forecasted values 

inv_testPred = scaler_l.inverse_transform(testPred)
print(inv_testPred[1])

# invert scaling for actual values

inv_y_valid = scaler_l.inverse_transform(y_valid)
print(inv_y_valid[1])


In [ ]:
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(inv_y_valid, inv_testPred))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# calculate Normalized RMSE
y_max = inv_y_valid.max()
y_min = inv_y_valid.min()
nrmse = rmse /(inv_y_valid.mean()) 
print('Test NRMSE:', nrmse)

In [ ]:
# calculate R-square
from sklearn.metrics import r2_score
from math import sqrt

r_sq = r2_score(inv_y_valid, inv_testPred)
print('Test R_Square: %.3f' % r_sq)

### Plotting

In [ ]:
from matplotlib import pyplot

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_train,trainPred)
ax.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=4)
ax.set_xlabel('observed')
ax.set_ylabel('predicted')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(inv_y_valid,inv_testPred) #[:,:,6]
ax.plot([inv_y_valid.min(), inv_y_valid.max()], [inv_y_valid.min(), inv_y_valid.max()], 'k--', lw=4)
ax.set_xlabel('observed')
ax.set_ylabel('predicted')
plt.show()

In [ ]:
trainPred = np.transpose(trainPred.flatten())
print(trainPred.shape)
print(y_train.shape)

In [ ]:
from matplotlib import pyplot

pyplot.plot(trainPred)
pyplot.plot(y_train)
pyplot.show()

In [ ]:
from datetime import datetime
lead_time = lag

time_range=pd.date_range(start="2019-01-01",end="2021-07-31",freq='m')
fig, ax= plt.subplots(figsize=(12, 4))

plt.plot(inv_y_valid/(10**6), color = 'red', label = 'Observed sea ice')
plt.plot(inv_testPred/(10**6), color = 'blue', label = 'LR_LSTM predictions')
#plt.title('Sea ice prediction (Lead time:'+str(lead_time)+' month)',fontsize = 15)
plt.xlabel('Month',fontsize = 10)
plt.ylabel('Sea ice extent ($10^6$ $Km^2$)',fontsize = 15)
#ax.grid(False)
#ax.set_facecolor('white')
time_idx=np.arange(0,30,3)
date_str=np.array(time_range[time_idx].strftime('%Y-%m'))
ax.set_xticks(time_idx)
ax.set_xticklabels(date_str)
plt.legend()
#plt.show()
fig.savefig('Time_series_sea_ice_prediction_attention_lead_time_'+str(lead_time)+'.png')